In [1]:
#!pip install gradio
#!pip install ipywidgets

In [1]:
import gradio as gr
import joblib
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model,Model

In [2]:
cnn = load_model("Model_2_CNN.h5")
svm = joblib.load("SVM_Classifier_2.h5")

extractCNN = Model(cnn.inputs, cnn.layers[-8].output)

classes = ['No DR', 'Mild DR','Moderate DR','Severe DR', 'Proliferative DR']

X = np.load('data.npy')
Y = np.load('label.npy')

print(X.shape)
print(Y.shape)

features_train  = extractCNN.predict(X)

TrainSVMScore=svm.score(features_train, Y)*100

print(TrainSVMScore)

In [3]:
def predict_input_image(img):
    
    print(img.size)
    img = img.resize((224,224))
    print(img.size)
    img_array = np.array(img)
    img_array = img_array.reshape((-1,224, 224, 3))
 
    img_array = img_array / 255.0
    print(img_array.shape)
        
    exfeature = extractCNN.predict(img_array)
    prediction = svm.predict(exfeature)
    
    print(prediction)
    return classes[prediction[0]]

In [4]:
gr.Interface(fn=predict_input_image,
             inputs=gr.Image(type="pil"),
             outputs="label",
             examples=["0b8bdec9d869.png"],
             allow_flagging = 'never').launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.7.1, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://0970595f6640121691.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
